# 第5章 Pythonを使った人工知能の前処理を学ぶ

## 5.1 はじめに

臨床での開発や研究が盛んにおこなわれていない。
臨床施設は画像データを含む多くの医療データを所有している。
SVNにはスケール変換が必要なことは有名。

---

## 5.2 テーブルデータにおける欠損値の取り扱い

### 5.2.1 テーブルデータの読み込み


リスト5.2.2 pandasを用いたcsvファイルの読み込みと表示

In [17]:
import pandas as pd
# コンマ区切りでcsvファイルを読み込み、性別を0, 1に置き換える。
df = pd.read_csv(
    "/Users/toku/Documents/Progects/カルー勉強会/"+
    "titanic/train.csv", sep=",").replace("male", 0).replace("female", 1)

print(df)

     PassengerId  Survived  Pclass  \
0              1         0       3   
1              2         1       1   
2              3         1       3   
3              4         1       1   
4              5         0       3   
..           ...       ...     ...   
886          887         0       2   
887          888         1       1   
888          889         0       3   
889          890         1       1   
890          891         0       3   

                                                  Name  Sex   Age  SibSp  \
0                              Braund, Mr. Owen Harris    0  22.0      1   
1    Cumings, Mrs. John Bradley (Florence Briggs Th...    1  38.0      1   
2                               Heikkinen, Miss. Laina    1  26.0      0   
3         Futrelle, Mrs. Jacques Heath (Lily May Peel)    1  35.0      1   
4                             Allen, Mr. William Henry    0  35.0      0   
..                                                 ...  ...   ...    ...   
886        

### 5.2.2 欠損値の処理

データの欠損３種類:
1. MCAR(missing completely at random)  
    完全にランダムな原因で欠損が発生
1. MAR (missing at random)  
    観測データに依存する原因で欠損が発生
1. MNAR(missing not at random)
    欠損値そのものに原因して欠損が発生

対処法:
1. 完全ケース分析  
    欠損値を含むサンプルを除外
1. 欠損値を補完

しかし、MARやMNARを多く含むデータセットの場合は、完全ケース分析を行うと、
解析結果に大きな歪みを生じてしまう。

欠損値に何らかの値を代入する方法として平均値代入法、回帰代入法などがある。
単純だが、学習う結果に基づく推定の際に、大きな誤差を生みかねない。
近年では、多重代入法が考案されている。

以降では、欠損値をもつデータを削除するリストワイズ削除処理と、シンプルな
欠損値補完を行う。

リスト5.2.3 データセット内のクラス別欠損値数表示

In [18]:
df.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

欠損値を処理するメソッド:
    1. dropna(): データ欠損のある行または列を削除
    1. fillna(): データ欠損のある要素を補完する

リスト5.2.4 データセットに対するリストワイズ処理

In [19]:
df_listwise = df.dropna()
df_listwise.isnull().sum()

PassengerId    0
Survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Ticket         0
Fare           0
Cabin          0
Embarked       0
dtype: int64

リスト5.2.5 データセットのデータを確認

In [20]:
print(df_listwise['Age'])

1      38.0
3      35.0
6      54.0
10      4.0
11     58.0
       ... 
871    47.0
872    33.0
879    56.0
887    19.0
889    26.0
Name: Age, Length: 183, dtype: float64


データ数の減少より、推定自体の悪化が予想される。

次に、平均値代入法を適用する。これは、単一代入法の一つである。代入モデルという欠損値を予測するモデルを作成し、作成されたモデルによって欠損値を補う。

リスト5.2.6 平均値代入法

In [23]:
df_mean = df.copy(deep=False) 
df_mean['Age'] = df_mean['Age'].fillna(df_mean['Age'].mean())
df_mean.isnull().sum()

PassengerId      0
Survived         0
Pclass           0
Name             0
Sex              0
Age              0
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

リスト5.2.7  Age列を確認

In [24]:
print(df_mean['Age'])

0      22.000000
1      38.000000
2      26.000000
3      35.000000
4      35.000000
         ...    
886    27.000000
887    19.000000
888    29.699118
889    26.000000
890    32.000000
Name: Age, Length: 891, dtype: float64


推定誤差の評価ができない